In [2]:
import numpy as np
from collections import Counter

In [251]:
def generate_lpn_problem(n, m, error_rate):
    s = np.random.randint(0, 2, n)  # Secret vector
#==========FOR NOW SECRET IS GENERATED WITHIN A PROBLEM INSTANCE=======================================
    A = np.random.randint(0, 2, (m, n))  #binary matrix
    noise = np.random.choice([0, 1], size=m, p=[1 - error_rate, error_rate])  
    b = (A @ s) % 2 
    b = (b + noise) % 2  # Adding noise
    return A, b, s

#=========EXAMPLE INSTANCE==============================================================================
# Parameters for the LPN problem
n = 8 # Number of dimensions
m = 1024  # Number of equations
error_rate = 0.2  # Probability of noise
A, b, s = generate_lpn_problem(n, m, error_rate)

print("Matrix A:\n", A)
print("\nNoisy vector b:\n", b)
print("\nSecret vector s:\n", s)


Matrix A:
 [[1 0 0 ... 1 0 0]
 [1 0 0 ... 1 0 0]
 [1 1 0 ... 0 1 1]
 ...
 [1 1 1 ... 0 0 0]
 [0 0 1 ... 1 0 1]
 [0 1 1 ... 0 1 0]]

Noisy vector b:
 [0 1 1 ... 0 0 0]

Secret vector s:
 [0 1 0 1 1 0 1 0]


In [252]:
def bkw_algo(A, b, bkt_sz):
    rows, cols = A.shape
    bkts = [np.zeros((0, cols), dtype=int) for _ in range(bkt_sz)]
    bkts_b = [np.zeros(0, dtype=int) for _ in range(bkt_sz)]
    idxs = np.random.permutation(rows)#shuffling the inputs
    A = A[idxs]
    b = b[idxs]
    
    #Bucket reduction
    for r in range(rows):
        reduced = False
        for _ in range(bkt_sz):  #randomly reduce using a bucket
            bkt_idx = np.random.randint(0, bkt_sz)  #random bucket index
            if bkts[bkt_idx].shape[0] > 0:  #if bucket is not empty
                vec_idx = np.random.randint(0, bkts[bkt_idx].shape[0])  #use a random vector to zero out
                if A[r, bkt_idx] == 1:
                    # Reduce the vector using random bucket vector
                    A[r] ^= bkts[bkt_idx][vec_idx]
                    b[r] ^= bkts_b[bkt_idx][vec_idx]
            if np.all(A[r][:bkt_idx] == 0):  #successfully reduced? check
                bkts[bkt_idx] = np.vstack([bkts[bkt_idx], A[r]])
                bkts_b[bkt_idx] = np.append(bkts_b[bkt_idx], b[r])
                reduced = True
                break
        if not reduced:
            #not reduced, place into the first bucket
            bkts[0] = np.vstack([bkts[0], A[r]])
            bkts_b[0] = np.append(bkts_b[0], b[r])
    
    #Gaussian elimination on reduced A and b
    red_A = []
    red_b = []
    for c in range(bkt_sz):
        red_A.append(bkts[c])
        red_b.append(bkts_b[c])
    red_A = np.vstack(red_A)
    red_b = np.concatenate(red_b)
    
    # Solve using code found on the internet
    num_cols = red_A.shape[1]
    for r in range(num_cols):
        # Find a pivot row
        pivot = None
        for c in range(r, red_A.shape[0]):
            if red_A[c, r] == 1:
                pivot = c
                break
        if pivot is None:
            continue  # Skip if no pivot found
        # Swap rows
        red_A[[r, pivot]] = red_A[[pivot, r]]
        red_b[[r, pivot]] = red_b[[pivot, r]]
        # Eliminate below
        for c in range(r + 1, red_A.shape[0]):
            if red_A[c, r] == 1:
                red_A[c] ^= red_A[r]
                red_b[c] ^= red_b[r]
    
    #back-substitution
    x = np.zeros(num_cols, dtype=int)
    for r in range(num_cols - 1, -1, -1):
        if red_A[r, r] == 1:
            x[r] = red_b[r]
            for c in range(r + 1, num_cols):
                x[r] ^= red_A[r, c] * x[c]
    return x


In [253]:
n = 4 # Number of dimensions
m = 128  # Number of equations
error_rate = 0.05  # Probability of noise
A, b, s = generate_lpn_problem(n, m, error_rate)
bucket_size = 10  # Set bucket size for reduction
bucket_size = 4  # Set bucket size for reduction
recovered_s = bkw_algo(A, b, bucket_size)

# Display results
print("Original secret vector s:\n", s)
print("\nRecovered secret vector using Randomized BKW:\n", recovered_s)
print("\nRecovery successful:", np.array_equal(s, recovered_s))


Original secret vector s:
 [0 0 1 1]

Recovered secret vector using Randomized BKW:
 [0 0 1 1]

Recovery successful: True


In [260]:
#==========IMPLEMENTING A MAJORITY BIT VOTING SYSTEM==========================================================
def majority_vote_bkw(A, b, bucket_size, runs=10):
    m, n = A.shape
    votes = np.zeros((runs, n), dtype=int)  # recovering vectors for votes

    # Run algorithm multiple times
    for i in range(runs):
        recovered_s = bkw_algo(A, b, bucket_size)
        votes[i] = recovered_s

    #majority voting for each bit
    final_s = np.zeros(n, dtype=int)
    for bit in range(n):
        bit_votes = votes[:, bit]
        final_s[bit] = 1 if np.sum(bit_votes) > runs / 2 else 0  #majority rule
    
    return final_s



In [266]:
bucket_size = 2  # Set bucket size for reduction
runs = 20  # Number of times to run BKW for majority voting

final_recovered_s = majority_vote_bkw(A, b, bucket_size, runs)

# Display results
print("Original secret vector s:\n", s)
print("\nRecovered secret vector using Majority Vote BKW:\n", final_recovered_s)
print("\nRecovery successful:", np.array_equal(s, final_recovered_s))


Original secret vector s:
 [0 0 1 1]

Recovered secret vector using Majority Vote BKW:
 [0 0 1 1]

Recovery successful: True


### Computing accuracy and sucess rates
Now we test the algorithm for the case its success rate as in the numebr of vectors it gets right completely and we also check its bit wise accuracy 

In [273]:
def compute_success_rate(n, m, error_rate, bucket_size, runs_per_instance, test_cases):
    successes = 0

    for test in range(test_cases):
        A, b, s = generate_lpn_problem(n, m, error_rate)
        recovered_s = majority_vote_bkw(A, b, bucket_size, runs_per_instance)
        success = np.array_equal(s, recovered_s)
        if success:#check if recovered is the same as initial secret
            successes += 1
        # Print progress
        print(f"Test Case {test + 1}/{test_cases}")
        print(f"  Secret:      {s}")
        print(f"  Recovered:   {recovered_s}")
        print(f"  Success:     {'Yes' if success else 'No'}\n")

    #Calculate success rate
    success_rate = successes / test_cases
    return success_rate


In [274]:
n = 4  # Secret vector size
m = 60 # Number of equations
error_rate = 0.25  # noise rate
bucket_size = 2  
runs_per_instance = 10  #number of runs for majority voting
test_cases = 100  
success_rate = compute_success_rate(n, m, error_rate, bucket_size, runs_per_instance, test_cases)


print(f"Success rate over {test_cases} test cases: {success_rate * 100:.2f}%")


Test Case 1/100
  Secret:      [1 0 1 1]
  Recovered:   [0 0 0 0]
  Success:     No

Test Case 2/100
  Secret:      [1 1 0 0]
  Recovered:   [0 1 0 0]
  Success:     No

Test Case 3/100
  Secret:      [0 1 1 1]
  Recovered:   [0 1 1 1]
  Success:     Yes

Test Case 4/100
  Secret:      [0 0 1 0]
  Recovered:   [0 0 0 1]
  Success:     No

Test Case 5/100
  Secret:      [0 0 0 0]
  Recovered:   [0 0 0 1]
  Success:     No

Test Case 6/100
  Secret:      [0 1 1 1]
  Recovered:   [1 0 1 0]
  Success:     No

Test Case 7/100
  Secret:      [0 1 0 0]
  Recovered:   [0 1 0 0]
  Success:     Yes

Test Case 8/100
  Secret:      [1 1 1 1]
  Recovered:   [0 0 1 0]
  Success:     No

Test Case 9/100
  Secret:      [0 0 1 0]
  Recovered:   [0 0 1 1]
  Success:     No

Test Case 10/100
  Secret:      [1 0 0 1]
  Recovered:   [0 0 0 0]
  Success:     No

Test Case 11/100
  Secret:      [1 1 1 1]
  Recovered:   [1 1 1 1]
  Success:     Yes

Test Case 12/100
  Secret:      [1 0 1 1]
  Recovered:   [0 

In [276]:
def compute_bitwise_accuracy(n, m, error_rate, bucket_size, runs_per_instance, test_cases):
    total_bits = n * test_cases  
    correct_bits = 0  # Count of correctly recovered bits

    for test in range(test_cases):
        # Generate a random LPN problem
        A, b, s = generate_lpn_problem(n, m, error_rate)
        recovered_s = majority_vote_bkw(A, b, bucket_size, runs_per_instance)
        correct_bits += np.sum(s == recovered_s)

        # Print progress
        print(f"Test Case {test + 1}/{test_cases}")
        print(f"  Secret:      {s}")
        print(f"  Recovered:   {recovered_s}")
        print(f"  Correct Bits: {np.sum(s == recovered_s)} / {n}\n")

    #bitwise accuracy
    bitwise_accuracy = (correct_bits / total_bits) * 100
    return bitwise_accuracy


In [278]:
# Parameters for the test
n = 4  
m = 128  
error_rate = 0.25 
bucket_size = 1  
runs_per_instance = 10 
test_cases = 100  

bitwise_accuracy = compute_bitwise_accuracy(n, m, error_rate, bucket_size, runs_per_instance, test_cases)

print(f"Bitwise accuracy over {test_cases} test cases: {bitwise_accuracy:.2f}%")


Test Case 1/100
  Secret:      [1 0 0 0]
  Recovered:   [1 0 0 0]
  Correct Bits: 4 / 4

Test Case 2/100
  Secret:      [0 1 1 1]
  Recovered:   [0 1 1 1]
  Correct Bits: 4 / 4

Test Case 3/100
  Secret:      [1 1 0 1]
  Recovered:   [1 1 1 1]
  Correct Bits: 3 / 4

Test Case 4/100
  Secret:      [0 1 1 1]
  Recovered:   [0 1 0 0]
  Correct Bits: 2 / 4

Test Case 5/100
  Secret:      [0 1 1 1]
  Recovered:   [0 1 1 1]
  Correct Bits: 4 / 4

Test Case 6/100
  Secret:      [1 1 0 1]
  Recovered:   [1 0 0 0]
  Correct Bits: 2 / 4

Test Case 7/100
  Secret:      [1 1 1 0]
  Recovered:   [1 1 1 0]
  Correct Bits: 4 / 4

Test Case 8/100
  Secret:      [0 1 1 1]
  Recovered:   [0 0 0 1]
  Correct Bits: 2 / 4

Test Case 9/100
  Secret:      [1 0 1 1]
  Recovered:   [1 0 0 0]
  Correct Bits: 2 / 4

Test Case 10/100
  Secret:      [0 1 0 0]
  Recovered:   [0 1 0 0]
  Correct Bits: 4 / 4

Test Case 11/100
  Secret:      [0 1 1 1]
  Recovered:   [0 1 1 1]
  Correct Bits: 4 / 4

Test Case 12/100
  